In [ ]:
# Implement the Modeling class in Home-Credit-Prediction/homecredit/model.py

In [22]:

import os
import sys
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency # need this for chi-squared function
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
path_dir = (os.path.dirname(os.getcwd()))
sys.path.append(path_dir)
    
from homecredit.data import HomeCredit
from homecredit.preparation import Preparation
from homecredit.cleaner import Cleaning
from homecredit.exploration import Exploration

In [4]:
#.remove_missvalues() method
# removes first some entries
# then, Replaces the NaNs in numerical column by the mean of values
# in numerical column respectively 
# for categorical variables, NaNs are replaced by ''
df = Cleaning().remove_missvalues()
df.shape

(91206, 122)

In [35]:
catcols = Cleaning().catcols # Preparation().get_catcols() # categorical columns
numcols = Cleaning().numcols # Preparation().get_numcols() # numerical columns

In [36]:
#data = df.drop_duplicates(subset = df.columns)
#data.shape
#There are no duplicates values

In [8]:
# N.B : Here, we use OneHotEncoder, but we also can use 
#pandas.get_dummies. More, get_dummies easier than OneHotEncoder

def encoding_categ_column(df, cols):
    
    for col_name in cols:
    
        L = list(df[col_name].unique())
        if '' in L:
            df[col_name].replace("", "NoValue", inplace=True) #Replace NaN by "NoCodeNature"

        ohe = OneHotEncoder(sparse = False) # Instanciate encoder
        ohe.fit(df[[col_name]]) # Fit encoder  ---> OneHotEncoder(sparse=False)

        col_encoded = ohe.transform(df[[col_name]]) # Encode

        dicts_col = {}
        keys = list(ohe.categories_[0])
        values = col_encoded.T.astype(int)

        for i,j in enumerate(keys):
            dicts_col[j] = values[i,:]

        result = pd.DataFrame.from_dict(dicts_col)

        df = df.reset_index(drop=True)

        #Concat df and result dataframes
        data_res = pd.concat([df, result], axis = 1)

        if 'NoValue' in list(data_res.columns):
            data_res = data_res.drop(columns= ['NoValue',col_name] )
            df = data_res
        else:
            data_res = data_res.drop(columns= col_name)
            df = data_res
        
    return df

In [9]:
encoded_df = encoding_categ_column(df = df,
                                   cols = catcols)
#encoded_df.shape

In [14]:
# create X, y
y = encoded_df.TARGET
X = encoded_df.drop('TARGET', axis = 1)

# Feature names
#features = list(X.columns)

# Split into Train/Test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# KNN model
neigh = KNeighborsClassifier(n_neighbors=5)

# Train the model on the Training data
neigh.fit(X_train, y_train)

# Score the model on the Testing data
score = neigh.score(X_test, y_test) 
print( "score : ", score )

score :  0.9194137855419925


In [15]:

# KNN model
neigh = KNeighborsClassifier(n_neighbors=5)

# Train the model on the Training data
neigh.fit(X_train, y_train)

# cross validation 
# Test model performance 
score = cross_val_score(neigh, X_train, y_train, cv=5).mean() 
print("score : ", score)

#  Predict on new data
y_pred = neigh.predict(X_test)

# Test accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))


score :  0.9214021659894115
Accuracy: 0.9194137855419925


In [17]:
scoring = ['roc_auc', 'accuracy']
models = []
results = []

# Classifiers
models.append(('LR', LogisticRegression()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('DTC', DecisionTreeClassifier()))
#models.append(('RF', RandomForestClassifier()))
#models.append(('SVC', SVC()))

In [18]:
for name, model in models:
    for s in scoring:
        model.fit(X_train, y_train)
        res = cross_val_score(model, X_train, y_train, cv=5, scoring=s).mean()
        results.append(res)
        print("Model: ", name, " scoring:", s, " score", res)

/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
   

Model:  LR  scoring: roc_auc  score 0.6130893675025983


/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
   

Model:  LR  scoring: accuracy  score 0.9258818373955877
Model:  KNN  scoring: roc_auc  score 0.5261843510796194
Model:  KNN  scoring: accuracy  score 0.9214021659894115
Model:  DTC  scoring: roc_auc  score 0.5319870075877831
Model:  DTC  scoring: accuracy  score 0.8626965840476357


In [20]:
model = LogisticRegression()
scoring = 'accuracy'

model.fit(X_train, y_train)

/Users/kenzaelhoussaini/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [21]:
#  Predict on new data
y_pred = model.predict(X_test)

# Test accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))   

Accuracy: 0.9239090709743439


In [30]:
# Scaling data

# create X, y
y = encoded_df.TARGET
X = encoded_df.drop('TARGET', axis = 1)

# Scaling features
scaler = MinMaxScaler() # Instanciate StandarScaler
scaler.fit(X)

X_rescaled = scaler.transform(X)

# Split into Train/Test
X_train_sc, X_test_sc, y_train, y_test = train_test_split(X_rescaled, y, test_size=0.3)


In [31]:
scoring = ['roc_auc', 'accuracy']
models = []
results = []

# Classifiers
models.append(('LR', LogisticRegression(max_iter=1000)))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('DTC', DecisionTreeClassifier()))

for name, model in models:
    for s in scoring:
        model.fit(X_train_sc, y_train)
        res = cross_val_score(model, X_train_sc, y_train, cv=5, scoring=s).mean()
        results.append(res)
        print("Model: ", name, " scoring:", s, " score", res)      

Model:  LR  scoring: roc_auc  score 0.7400404423400226
Model:  LR  scoring: accuracy  score 0.9253962741845608
Model:  KNN  scoring: roc_auc  score 0.5481678467183916
Model:  KNN  scoring: accuracy  score 0.9220443474965118
Model:  DTC  scoring: roc_auc  score 0.5361585969728132
Model:  DTC  scoring: accuracy  score 0.8607700002708627


**LogisticRegression model gives the best score**

**Let's make predictions on test data**

In [32]:
model = LogisticRegression(max_iter=1000)
scoring = 'accuracy'

model.fit(X_train_sc, y_train)
res = cross_val_score(model, X_train_sc, y_train, cv=5, scoring=s).mean()
print("score: ", res)  

#  Predict on new data
y_pred = model.predict(X_test_sc)

# Test accuracy
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))   

score:  0.9253962741845608
Accuracy: 0.9250420290914407


In [ ]:
class Encoder():
    
    """ Initialize dataframe
    """
    def __init__(self, df):

        self.df = df
        
    def execute_list(self, cols):
        
        for col_name in cols:
    
        L = list(df[col_name].unique())
        if '' in L:
            df[col_name].replace("", "NoValue", inplace=True) #Replace NaN by "NoCodeNature"

        ohe = OneHotEncoder(sparse = False) # Instanciate encoder
        ohe.fit(df[[col_name]]) # Fit encoder  ---> OneHotEncoder(sparse=False)

        col_encoded = ohe.transform(df[[col_name]]) # Encode

        dicts_col = {}
        keys = list(ohe.categories_[0])
        values = col_encoded.T.astype(int)

        for i,j in enumerate(keys):
            dicts_col[j] = values[i,:]

        result = pd.DataFrame.from_dict(dicts_col)

        df = df.reset_index(drop=True)

        #Concat df and result dataframes
        data_res = pd.concat([df, result], axis = 1)

        if 'NoValue' in list(data_res.columns):
            data_res = data_res.drop(columns= ['NoValue',col_name] )
            df = data_res
        else:
            data_res = data_res.drop(columns= col_name)
            df = data_res
        
        return df
        
 

In [ ]:

class Modeling:
    
    def __init__(self):
            
        path_dir = (os.path.dirname(os.getcwd()))
        sys.path.append(path_dir)
        # Assign an attribute ".data" to all new instances of Preparation
        self.data = Cleaning().remove_entries() # good practice to be sure not to modify your `data` variable
        self.catcols = Preparation().get_catcols()
        self.numcols = Preparation().get_numcols()
   